## Milestone project #1: Food Vision Big

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import sklearn
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import os 
import pathlib 
import random
import matplotlib.image as mpimg
import zipfile
import wget
import tensorflow_hub as hub
import tensorflow_datasets as tfds

c:\anaconda3\envs\tf2.4\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get Helper Functions

In [2]:
import requests
url = "https://raw.githubusercontent.com/Meta-c/Helper-ML/master/ML_functions.py"


response = requests.get(url)
code = response.text

exec(code)

## Use TensorFlow datasets

In [6]:
# List all available datasets
datasets_list=tfds.list_builders()
print("food101"in datasets_list)

True


In [7]:
# Load in the data 
(train_data,test_data), ds_info = tfds.load(name="food101",
                                            split=["train","validation"],
                                            shuffle_files=True,
                                            as_supervised=True,
                                            with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [04:36<?, ? url/s]
